Check if all files are uploaded successfully.

In [0]:
display(dbutils.fs.ls("/FileStore/tables/"))

Merge the files and preview the dataset.

In [0]:
# Load all CSV files while letting Databricks handle headers automatically
crime_df = spark.read.option("header", "true").csv("/FileStore/tables/CrimeDataset_*.csv", inferSchema=True)

In [0]:
# Show first few rows to verify correct loading
crime_df.show(5)

In [0]:
# Check total number of records
print(f"Total records in dataset: {crime_df.count()}")

To perform dataset cleaning, the first is to check for missing values of each column.


In [0]:
from pyspark.sql.functions import col, sum

# Count missing values per column
missing_values = crime_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in crime_df.columns])

# Show the result
missing_values.show()

For better readability, I converted the number of missing values of each column into table, which can further be visualized for even better readability. In the table, I can easily identify the attributes(columns) that matters to my future analysis.

In [0]:
import pandas as pd

# Convert Spark DataFrame to Pandas
missing_values_pd = missing_values.toPandas()

# Transpose the DataFrame to have columns as index
missing_values_pd = missing_values_pd.T  # Flip rows and columns

# Create a new column for proper labels
missing_values_pd.reset_index(inplace=True)
missing_values_pd.columns = ["Column Name", "Missing Count"]

# Sort by the highest number of missing values
missing_values_pd = missing_values_pd.sort_values(by="Missing Count", ascending=False)

# Display as a table
display(missing_values_pd)

I identify that "Vict Sex" is one of the column that I would be using. Thus, I first want to know all its values and then decide what to do with the missing values.

In [0]:
crime_df.groupBy("Vict Sex").count().orderBy("count", ascending=False).show()

As the originally in the column there is already an "X", meaing "sex unknown", thus I decided to transform all sex other than "M" and "F", including "null", into "X".

In [0]:
from pyspark.sql.functions import when

# Standardize 'Vict Sex' column
crime_df = crime_df.withColumn(
    "Vict Sex",
    when(crime_df["Vict Sex"].isin("M", "F", "X"), crime_df["Vict Sex"])
    .otherwise("X")  # Replace invalid values with 'X' (Unknown)
)

crime_df.groupBy("Vict Sex").count().orderBy("count", ascending=False).show()

Next I also identify "Vict Descent" as one of the attribute I want to work with, therefore I want to check the counts of each value and how the "null" value.

In [0]:
crime_df.groupBy("Vict Descent").count().orderBy("count", ascending=False).show()

Similar to "Vict Sex", "Vict Descent" already has a "X" value that represents "unknown". Thus I decided to transform all null values and other values not listed in the original dataset explanation into "X".

In [0]:
# List of valid descent codes
valid_descents = ["A", "B", "C", "D", "F", "G", "H", "I", "J", "K", "L", "O", "P", "S", "U", "V", "W", "X", "Z"]

In [0]:
# Standardize 'Vict Descent' column
crime_df = crime_df.withColumn(
    "Vict Descent",
    when(crime_df["Vict Descent"].isin(valid_descents), crime_df["Vict Descent"])
    .otherwise("X")  # Replace invalid values with 'X' (Unknown)
)
crime_df.groupBy("Vict Descent").count().orderBy("count", ascending=False).show()

After performing data cleaning, I want to start the analysis. First is to see the overall top crimes occurred. I use plotly to create a bar chart that shows the Top 10 crimes occurred during the period of the dataset (2020-2024)

In [0]:
import plotly.express as px
import pandas as pd

# Aggregate top 10 crimes
top_crimes_df = crime_df.groupBy("Crm Cd Desc").count().orderBy("count", ascending=False).limit(10)

# Convert to Pandas DataFrame for Plotly
top_crimes_pd = top_crimes_df.toPandas()

# Create a horizontal bar chart
fig = px.bar(
    top_crimes_pd, 
    x="count",  # Horizontal bars
    y="Crm Cd Desc",  # Crime Types on Y-axis
    title="Top 10 Crimes Occurred Overall",
    labels={"Crm Cd Desc": "Crime Type", "count": "Number of Occurrences"},
    color="Crm Cd Desc",  # Assign different colors for each crime type
    color_discrete_sequence=px.colors.qualitative.Set3,  # Color palette
    orientation="h"  # Makes the chart horizontal
)

# Remove legend
fig.update_layout(showlegend=False)

# Improve layout
fig.update_layout(
    xaxis_title="Number of Occurrences",
    yaxis_title="Crime Type"
)

# Show the plot
fig.show()


Next, I want to see the trend of the top 10 crimes of each year (2020, 2021, 2022, 2023 and 2024). So I extracted the "year" from "DATE OCC" column and created a new column name "Year" to the dataset.

In [0]:
from pyspark.sql.functions import year, to_date

# Convert "DATE OCC" to a proper date format and extract the year again
crime_df = crime_df.withColumn("Year", year(to_date(crime_df["DATE OCC"], "MM/dd/yyyy hh:mm:ss a")))

# Check extracted years
crime_df.select("DATE OCC", "Year").distinct().show(20)

And then we make sure there no "null" value in the new column.

In [0]:
crime_df = crime_df.filter(crime_df["Year"].isNotNull())

Next, I aggregate top 10 crime trends over years

In [0]:
from pyspark.sql.functions import col, count, desc

# Compute the top 10 crimes list
top_crimes_df = (
    crime_df.groupBy("Crm Cd Desc")
    .agg(count("*").alias("count"))
    .orderBy(desc("count"))
    .limit(10)
)

# Convert to list
top_crimes_list = [row["Crm Cd Desc"] for row in top_crimes_df.collect()]

# Check if list is empty before proceeding
if not top_crimes_list:
    raise ValueError("Error: `top_crimes_list` is empty. Check if `crime_df` contains valid crime descriptions.")

# Filter dataset for only top 10 crimes
crime_trend_df = crime_df.filter(col("Crm Cd Desc").isin(top_crimes_list))

# Group by Year and Crime Type
crime_trend_df = crime_trend_df.groupBy("Year", "Crm Cd Desc").count()

# Convert to Pandas for Plotly
crime_trend_pd = crime_trend_df.toPandas()

# Ensure Year column is sorted correctly
crime_trend_pd = crime_trend_pd.sort_values(by=["Year", "count"], ascending=[True, False])

display(crime_trend_pd)

Using plotly, I create a line chart showing the trend of the top 10 crimes over the 5 year period.

In [0]:
import plotly.express as px

# Convert Year column to an integer type (if necessary)
crime_trend_pd["Year"] = crime_trend_pd["Year"].astype(int)

# Create a line chart for crime trends
fig = px.line(
    crime_trend_pd,
    x="Year",
    y="count",
    color="Crm Cd Desc",
    title="Crime Trends Over Time (2020-2024)",
    labels={"count": "Number of Crimes", "Year": "Year", "Crm Cd Desc": "Crime Type"},
    markers=True
)

# Improve layout to ensure clear year labeling
fig.update_layout(
    xaxis=dict(dtick=1),  # Ensure year labels are displayed properly
    yaxis_title="Number of Occurrences",
    xaxis_title="Year"
)

# Show the plot
fig.show()

Next I want to see the top 10 crime of each year, and see if they are identical or each year have different top 10 crimes. So first, i computed the top 10 crimes of each year.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank

# Define a window function to rank crimes within each year
window_spec = Window.partitionBy("Year").orderBy(col("count").desc())

# Rank crimes per year
crime_trend_ranked = crime_trend_df.withColumn("rank", rank().over(window_spec))

# Keep only the top 10 crimes per year
top_crimes_per_year = crime_trend_ranked.filter(col("rank") <= 10)

# Convert to Pandas for Plotly
top_crimes_per_year_pd = top_crimes_per_year.toPandas()

Next, I visualize the top 10 crime of each year using separate bar charts.

In [0]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get unique years
unique_years = sorted(top_crimes_per_year_pd["Year"].unique())

# Create a subplot with a row for each year
fig = make_subplots(
    rows=len(unique_years), 
    cols=1, 
    subplot_titles=[f"Top 10 Crimes in {year}" for year in unique_years]
)

# Assign consistent colors for each crime type
color_map = {crime: px.colors.qualitative.Dark24[i % 24] for i, crime in enumerate(top_crimes_per_year_pd["Crm Cd Desc"].unique())}

# Add bar charts for each year, making sure to sort within each year before plotting
for i, year in enumerate(unique_years):
    year_df = top_crimes_per_year_pd[top_crimes_per_year_pd["Year"] == year]
    
    # Sort the top 10 crimes within each year by count (descending) before plotting
    year_df = year_df.sort_values(by="count", ascending=True)  # Important: ascending=True for horizontal bars

    fig.add_trace(
        go.Bar(
            x=year_df["count"],
            y=year_df["Crm Cd Desc"],
            orientation='h',
            marker=dict(color=[color_map[crime] for crime in year_df["Crm Cd Desc"]]),  # Assign consistent colors
            name=f"{year}"
        ),
        row=i+1, col=1
    )

# Update layout for better visualization
fig.update_layout(
    title="Top 10 Most Reported Crimes Per Year",
    xaxis_title="Number of Crimes",
    yaxis_title="Crime Type",
    height=350 * len(unique_years),  # Dynamically adjust height
    showlegend=False  # Hide repeated legends
)

# Show the plot
fig.show()

Next, I want to create a table that could show details of each crime type, including the total occurence, occurence each year, most occurred area, most common victim sex and victim race. I used "THEFT OF IDENTITY" as the example. But simply by changing the crime_type in step 1, I can access to the details of each crime type.

In [0]:
from pyspark.sql.functions import col, count, desc
import pandas as pd

# 🔹 Step 1: Define the Crime Type
crime_type = "THEFT OF IDENTITY"  # 🔹 Change this to analyze any crime

# 🔹 Step 2: Filter dataset for the selected crime type
crime_specific_df = crime_df.filter(col("Crm Cd Desc") == crime_type)

# 🔹 Step 3: Compute Metrics
# 1️⃣ Total occurrences overall
total_occurrences = crime_specific_df.count()

# 2️⃣ Occurrences per year
yearly_counts_df = (
    crime_specific_df.groupBy("Year")
    .agg(count("*").alias("count"))
    .orderBy("Year")
)

# Convert yearly counts to dictionary for easier access
yearly_counts_dict = {row["Year"]: row["count"] for row in yearly_counts_df.collect()}

# 3️⃣ Most affected area
most_affected_area = (
    crime_specific_df.groupBy("AREA NAME")
    .agg(count("*").alias("count"))
    .orderBy(desc("count"))
    .limit(1)
)

most_affected_area = most_affected_area.collect()[0]["AREA NAME"] if most_affected_area.count() > 0 else "N/A"

# 4️⃣ Most frequent victim racial identity
most_common_race = (
    crime_specific_df.groupBy("Vict Descent")
    .agg(count("*").alias("count"))
    .orderBy(desc("count"))
    .limit(1)
)

most_common_race = most_common_race.collect()[0]["Vict Descent"] if most_common_race.count() > 0 else "N/A"

# 5️⃣ Most frequent victim sex identity
most_common_sex = (
    crime_specific_df.groupBy("Vict Sex")
    .agg(count("*").alias("count"))
    .orderBy(desc("count"))
    .limit(1)
)

most_common_sex = most_common_sex.collect()[0]["Vict Sex"] if most_common_sex.count() > 0 else "N/A"

# 🔹 Step 4: Create the Summary Table
crime_summary = {
    "Crime Type": [crime_type],
    "Total Occurrences": [total_occurrences],
    "2020 Occurrences": [yearly_counts_dict.get(2020, 0)],
    "2021 Occurrences": [yearly_counts_dict.get(2021, 0)],
    "2022 Occurrences": [yearly_counts_dict.get(2022, 0)],
    "2023 Occurrences": [yearly_counts_dict.get(2023, 0)],
    "2024 Occurrences": [yearly_counts_dict.get(2024, 0)],
    "Most Affected Area": [most_affected_area],
    "Most Common Victim Race": [most_common_race],
    "Most Common Victim Sex": [most_common_sex],
}

# Convert to Pandas DataFrame
crime_summary_df = pd.DataFrame(crime_summary)

# 🔹 Step 5: Display the Table (Use the appropriate display function)
display(crime_summary_df)  # Use this in Databricks to display the table

This cell loads a CSV file from the Databricks FileStore into a Spark DataFrame.

1. **Defining File Path**  
   - The `folder_path` variable specifies the directory containing the CSV files.

2. **CSV Options**  
   - `infer_schema = "false"` prevents Spark from automatically inferring data types.  
   - `first_row_is_header = "false"` indicates that the first row is not treated as column headers.  
   - `delimiter = ","` specifies that the CSV file uses commas to separate values.

3. **Loading the CSV File into a DataFrame**  
   - `spark.read.format("csv")` specifies that the file format is CSV.  
   - `.option("inferSchema", infer_schema)` prevents automatic type inference.  
   - `.option("header", first_row_is_header)` ensures Spark does not treat the first row as column names.  
   - `.option("sep", delimiter)` sets the delimiter to a comma.  
   - `.load(folder_path)` reads the CSV data into a Spark DataFrame.

4. **Extracting Column Names from the First Row**  
   - `anoop_df.first()` retrieves the first row of the DataFrame, assuming it contains column headers.  
   - `anoop_df.filter(df["_c0"] != new_header[0])` removes the first row, as it is used for column names.

5. **Renaming Columns**  
   - `anoop_df.toDF(*[str(col) for col in new_header])` renames the columns using the extracted header values.

6. **Displaying the Data**  
   - `display(anoop_df)` prints the DataFrame in Databricks, showing a preview of the data.

This approach ensures the dataset is loaded correctly with meaningful column names instead of default `_c0`, `_c1`, etc.

In [0]:

folder_path = "/FileStore/tables/"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = "," 

anoop_df = spark.read.format("csv") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(folder_path)

# Extract the first row as header
new_header = anoop_df.first()  # Get first row
anoop_df = anoop_df.filter(anoop_df["_c0"] != new_header[0])  # Remove first row

# Rename columns using extracted header
anoop_df = anoop_df.toDF(*[str(col) for col in new_header])


display(anoop_df)

This cell processes the "TIME OCC" (Time of Occurrence) column to ensure it contains only valid numerical values in the HHMM (hour-minute) format.

1. **Importing Required Functions**
   - The `pyspark.sql.functions` module provides functions for data manipulation.
   - `col`: Used to refer to DataFrame columns.
   - `regexp_extract`: Extracts substrings from a column using regular expressions.
   - Other functions (`count, when, floor, concat, lit, lpad, to_date, dayofweek, to_timestamp, date_format`) are imported but not yet used in this block.
   - `plotly.express as px` is imported for visualization purposes but not used in this step.

2. **Extracting Valid Time Values**
   - `anoop_df.withColumn("TIME OCC", ...)` creates a new DataFrame (`df_time`) where the "TIME OCC" column is processed.
   - `col("TIME OCC").cast("string")` ensures the column is treated as a string.
   - `regexp_extract(col("TIME OCC"), r"(\d{3,4})", 0)`:  
     - This extracts 3 or 4-digit numbers from "TIME OCC".
     - The regular expression `(\d{3,4})` captures valid time values in HHMM format.
     - This helps clean the dataset and remove any non-numeric values or incorrectly formatted time values.

By performing this transformation, we ensure that the "TIME OCC" column contains only properly formatted time values, which is essential for time-based analysis.

In [0]:
from pyspark.sql.functions import col, count, regexp_extract, when, floor, concat, lit, lpad, to_date, dayofweek,to_timestamp, date_format
import plotly.express as px

# Ensure TIME OCC contains only numbers, extracting valid HHMM values (3 or 4-digit numbers)
df_time = anoop_df.withColumn("TIME OCC", regexp_extract(col("TIME OCC").cast("string"), r"(\d{3,4})", 0))

This cell ensures that the "TIME OCC" column contains only valid integer values by handling invalid entries and null values.

1. **Safely Converting "TIME OCC" to an Integer**
   - `df_time.withColumn("TIME OCC", when(col("TIME OCC") != "", col("TIME OCC").cast("int")).otherwise(None))`:
     - Checks if the column is not empty (`col("TIME OCC") != ""`).
     - Converts non-empty values to integers using `.cast("int")`.
     - If the value is empty or invalid, it is replaced with `None` (NULL), ensuring that invalid data does not cause errors.

2. **Handling Missing Values**
   - `df_time.dropna(subset=["TIME OCC"])`:
     - Removes rows where "TIME OCC" is NULL.
     - This ensures that all time values are valid and prevents issues in subsequent time-based analysis.

By performing this transformation, we ensure that:
- All values in "TIME OCC" are integers.
- Any invalid or empty values are handled safely.
- The dataset remains clean for further processing.

In [0]:
# Convert TIME OCC to integer safely (invalid values become NULL)
df_time = df_time.withColumn("TIME OCC", when(col("TIME OCC") != "", col("TIME OCC").cast("int")).otherwise(None))
# Replace NULL values with a default safe time (e.g., 0000 for midnight)
df_time = df_time.dropna(subset = ["TIME OCC"])

This cell processes the "TIME OCC" column to extract hour-based information and analyze crime distribution across different hours of the day.

1. **Extracting the Hour from "TIME OCC" (HHMM Format)**
   - `df_time.withColumn("Hour", floor(col("TIME OCC") / 100).cast("int"))`:
     - Since "TIME OCC" is in HHMM format (e.g., 2130 for 9:30 PM), dividing by 100 extracts only the hour.
     - `floor()` ensures that only the integer part (hour) remains.
     - `.cast("int")` converts the extracted value to an integer for further processing.

2. **Formatting Hours with Leading Zeros**
   - `df_hour.withColumn("Hour Formatted", lpad(col("Hour").cast("string"), 2, "0"))`:
     - Ensures that single-digit hours (e.g., 9) are displayed as two-digit values (e.g., "09").
     - `lpad()` adds a leading zero where necessary.

3. **Creating an Hourly Time Bin Label**
   - `df_hour.withColumn("Hour Bin", concat(col("Hour Formatted").cast("string"), lit(":00 - "), col("Hour Formatted").cast("string"), lit(":59")))`:
     - Creates a time range for each hour, such as "00:00 - 00:59" or "23:00 - 23:59".
     - `concat()` combines formatted hour strings with text labels to create readable bin labels.

4. **Counting Crimes per Hour**
   - `crime_by_hour = df_hour.groupBy("Hour Bin").agg(count("*").alias("Crime_Count")).orderBy("Hour Bin")`:
     - Groups crimes based on hourly time bins.
     - Uses `count("*")` to count the number of crimes in each time bin.
     - `orderBy("Hour Bin")` ensures that the results are sorted in chronological order.

This transformation enables analysis of crime trends across different hours of the day, helping to identify peak crime periods.

In [0]:
# Extract hour from TIME OCC (HHMM format)
df_hour = df_time.withColumn("Hour", floor(col("TIME OCC") / 100).cast("int"))

# Ensure hours are formatted correctly with leading zeros
df_hour = df_hour.withColumn("Hour Formatted", lpad(col("Hour").cast("string"), 2, "0"))

# Create a bin label (e.g., "00:00-00:59", "01:00-01:59", ...)
df_hour = df_hour.withColumn("Hour Bin", concat(
    col("Hour Formatted").cast("string"),
    lit(":00 - "),
    col("Hour Formatted").cast("string"),
    lit(":59")
))

# Count crimes per hour
crime_by_hour = df_hour.groupBy("Hour Bin").agg(count("*").alias("Crime_Count")).orderBy("Hour Bin")

This cell displays the summarized crime count per hour using the Databricks `display()` function.

1. **Displaying the Results**
   - `display(crime_by_hour)`:  
     - This function renders the `crime_by_hour` DataFrame as an interactive table in Databricks.
     - The table contains two columns:
       - **Hour Bin**: Represents the time range (e.g., "00:00 - 00:59", "01:00 - 01:59").
       - **Crime_Count**: The total number of crimes that occurred within that hour bin.

2. **Insights from the Table**
   - The table allows easy identification of peak crime hours.
   - Crime counts are highest during midday (e.g., 12:00 - 12:59) and lower in the early morning hours.
   - This information can be useful for law enforcement and city planning to allocate resources effectively.

The `display()` function in Databricks is particularly useful for interactive exploration, filtering, and visualization.

In [0]:
# Display results
display(crime_by_hour)

This cell creates an interactive bar chart using Plotly to visualize crime frequency by hour.

1. **Converting DataFrame to Pandas**
   - `crime_by_hour_pd = crime_by_hour.toPandas()`:
     - Converts the Spark DataFrame into a Pandas DataFrame.
     - This step is necessary because Plotly requires Pandas for visualization in Databricks.

2. **Creating a Bar Chart with Plotly**
   - `px.bar(...)` generates a bar chart with:
     - `x="Hour Bin"`: The hourly time bins (e.g., "00:00 - 00:59").
     - `y="Crime_Count"`: The total number of crimes in each hour.
     - `title="Crime Frequency by Hour of the Day"`: The title of the chart.
     - `labels={"Hour Bin": "Hour of the Day", "Crime_Count": "Crime Frequency"}`: Axis labels for clarity.
     - `text_auto=True`: Displays crime counts on top of each bar.

3. **Customizing the Chart Layout**
   - `fig.update_layout(...)` applies several formatting enhancements:
     - `xaxis_tickangle=-45`: Rotates x-axis labels by 45 degrees for better readability.
     - `bargap=0.1`: Adds spacing between bars.
     - `template="plotly_dark"`: Uses a dark theme, which is well-suited for Databricks.

4. **Displaying the Chart**
   - `fig.show()`: Renders the interactive visualization.

### **Insights from the Bar Chart**
- The highest crime occurrences are around **midday (12:00 - 12:59)**.
- Crime rates are significantly lower during early morning hours.
- The chart provides a clear hour-wise crime distribution, helping in strategic law enforcement and resource allocation.

This interactive visualization allows further exploration, such as zooming and hovering over bars to see precise values.

In [0]:
# If you want to visualize in Databricks, convert to Pandas and use matplotlib
crime_by_hour_pd = crime_by_hour.toPandas()

# Plot interactive bar chart using Plotly
fig = px.bar(
    crime_by_hour_pd,
    x="Hour Bin",
    y="Crime_Count",
    title="Crime Frequency by Hour of the Day",
    labels={"Hour Bin": "Hour of the Day", "Crime_Count": "Crime Frequency"},
    text_auto=True,  # Display count on bars
)

# Customize layout
fig.update_layout(
    xaxis_tickangle=-45,  # Rotate x-axis labels for readability
    bargap=0.1,  # Add some space between bars
    template="plotly_dark"  # Dark theme for Databricks compatibility
)

# Show interactive Plotly graph
fig.show()

This cell processes the "DATE OCC" column to extract the day of the week from crime occurrence dates.

1. **Ensuring "DATE OCC" is a String**
   - `anoop_df.withColumn("DATE OCC", col("DATE OCC").cast("string"))`:
     - Ensures that the "DATE OCC" column is treated as a string before conversion.
     - This step prevents potential formatting issues when converting to a timestamp.

2. **Converting "DATE OCC" to a Timestamp**
   - `df_day.withColumn("Date", to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a"))`:
     - Converts the date string into a Spark timestamp format.
     - The provided format `"MM/dd/yyyy hh:mm:ss a"` matches the structure of the dataset.

3. **Extracting the Day of the Week**
   - `df_day.withColumn("DayOfWeekNum", dayofweek(col("Date")))`:
     - Uses Spark's `dayofweek()` function to extract the day number (1 to 7).
     - In Spark, **1 = Sunday** and **7 = Saturday**.

4. **Mapping Numeric Days to Actual Names**
   - `.withColumn("DayOfWeek", when(col("DayOfWeekNum") == 1, "Sunday")...otherwise(None))`:
     - Converts numeric day representations into human-readable day names.
     - This makes the dataset easier to interpret and analyze.

5. **Displaying Sample Data**
   - `df_day.select("DATE OCC", "Date", "DayOfWeekNum", "DayOfWeek").show(10, truncate=False)`:
     - Selects relevant columns and displays the first 10 rows for verification.
     - The output confirms that each date is correctly mapped to its corresponding weekday.

### **Insights from the Output**
- The transformation successfully maps dates to their respective weekday names.
- This information is useful for analyzing crime trends by day of the week.
- For example, if crime rates are higher on weekends, law enforcement agencies can allocate more resources accordingly.

This step is crucial for understanding temporal patterns in crime data.

In [0]:
# Ensure DATE OCC is treated as a string before conversion
df_day = anoop_df.withColumn("DATE OCC", col("DATE OCC").cast("string"))

# Convert to timestamp using the detected format
df_day = df_day.withColumn("Date", to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a"))

# Extract day of the week (1 = Sunday, 7 = Saturday)
df_day = df_day.withColumn("DayOfWeekNum", dayofweek(col("Date")))

# Map numeric days to actual names
df_day = df_day.withColumn(
    "DayOfWeek",
    when(col("DayOfWeekNum") == 1, "Sunday")
    .when(col("DayOfWeekNum") == 2, "Monday")
    .when(col("DayOfWeekNum") == 3, "Tuesday")
    .when(col("DayOfWeekNum") == 4, "Wednesday")
    .when(col("DayOfWeekNum") == 5, "Thursday")
    .when(col("DayOfWeekNum") == 6, "Friday")
    .when(col("DayOfWeekNum") == 7, "Saturday")
)

# Show results to verify
df_day.select("DATE OCC", "Date", "DayOfWeekNum", "DayOfWeek").show(10, truncate=False)

This cell aggregates crime data by the day of the week and visualizes it using an interactive bar chart.

1. **Counting Crimes for Each Day of the Week**
   - `crime_by_day = df_day.groupBy("DayOfWeek").agg(count("*").alias("Crime_Count"))`:
     - Groups the dataset by "DayOfWeek" (e.g., Monday, Tuesday).
     - Uses `count("*")` to count the number of crimes occurring on each day.

   - `.orderBy("DayOfWeekNum")`:
     - Ensures that the days appear in the correct weekday order (Monday–Sunday).

2. **Dropping Numeric Day Column (Cleanup)**
   - `crime_by_day = crime_by_day.drop("DayOfWeekNum")`:
     - Removes the numeric representation of days since it's no longer needed for visualization.

3. **Converting DataFrame to Pandas for Visualization**
   - `crime_by_day_pd = crime_by_day.toPandas()`:
     - Converts the Spark DataFrame into a Pandas DataFrame for compatibility with Plotly.

4. **Creating a Bar Chart with Plotly**
   - `px.bar(...)` generates a bar chart with:
     - `x="DayOfWeek"`: The weekday names (e.g., Monday, Tuesday).
     - `y="Crime_Count"`: The total number of crimes on each day.
     - `title="Crime Frequency by Day of the Week"`: The chart title.
     - `labels={"DayOfWeek": "Day of the Week", "Crime_Count": "Crime Frequency"}`: Provides axis labels.
     - `text_auto=True`: Displays crime counts on top of bars.
     - `color="DayOfWeek"`: Uses different colors for each day for better visualization.

5. **Customizing Chart Layout**
   - `fig.update_layout(...)` applies additional formatting:
     - `xaxis_tickangle=-45`: Rotates x-axis labels by 45 degrees for better readability.
     - `bargap=0.1`: Adds spacing between bars.
     - `template="plotly_dark"`: Uses a dark theme for Databricks compatibility.

6. **Displaying the Chart**
   - `fig.show()`: Renders the interactive visualization.

### **Insights from the Output**
- **Friday** has the highest number of reported crimes.
- **Tuesday and Sunday** have slightly lower crime rates compared to other days.
- The consistent crime distribution throughout the week suggests no extreme weekday patterns, but weekends might see an increase.

This visualization helps law enforcement and policymakers understand when crimes are most frequent, assisting in strategic planning.

In [0]:
# Count crimes per day of the week
crime_by_day = df_day.groupBy("DayOfWeek", "DayOfWeekNum") \
    .agg(count("*").alias("Crime_Count")) \
    .orderBy("DayOfWeekNum")  # Ensures correct weekday order (Monday–Sunday)

# Drop numeric day column before visualization
crime_by_day = crime_by_day.drop("DayOfWeekNum")

# Convert to Pandas for visualization
crime_by_day_pd = crime_by_day.toPandas()

# Plot interactive bar chart using Plotly
fig = px.bar(
    crime_by_day_pd,
    x="DayOfWeek",
    y="Crime_Count",
    title="Crime Frequency by Day of the Week",
    labels={"DayOfWeek": "Day of the Week", "Crime_Count": "Crime Frequency"},
    text_auto=True,  # Display count on bars
    color="DayOfWeek",  # Different colors for each day
)

# Customize layout
fig.update_layout(
    xaxis_tickangle=-45,  # Rotate labels for readability
    bargap=0.1,  # Add space between bars
    template="plotly_dark"  # Dark theme for Databricks compatibility
)

# Show interactive Plotly graph
fig.show()

This cell extracts the year and month from the "DATE OCC" column and aggregates crime counts per month and year.

1. **Extracting Year and Month from "DATE OCC"**
   - `df_day.withColumn("Year", date_format(col("Date"), "yyyy").cast("int"))`:
     - Extracts the **year** from the "Date" column and converts it into an integer.

   - `df_day.withColumn("Month", date_format(col("Date"), "MMMM"))`:
     - Extracts the **full month name** (e.g., "January", "February") for better readability in visualizations.

   - `df_day.withColumn("MonthNum", date_format(col("Date"), "MM").cast("int"))`:
     - Extracts the **numeric month** (1–12) for sorting purposes.
     - This helps maintain chronological order when analyzing trends.

2. **Counting Crimes per Month and Year**
   - `crime_trends = df_day.groupBy("Year", "Month", "MonthNum").agg(count("*").alias("Crime_Count"))`:
     - Groups crimes by **year and month**.
     - Counts the total number of crimes in each month.

   - `.orderBy("Year", "MonthNum")`:
     - Ensures that months are displayed in proper chronological order (January → December) for each year.

3. **Dropping "MonthNum" After Sorting**
   - `crime_trends = crime_trends.drop("MonthNum")`:
     - The "MonthNum" column is no longer needed after sorting.
     - This keeps the dataset clean for visualization.

### **Why This Step is Important?**
- It allows analysis of crime trends over time (monthly and yearly patterns).
- Helps in identifying **seasonal variations** or **anomalies** in crime rates.
- Prepares the data for visualization in the next steps.

By structuring the data this way, we can generate time-series plots to see how crime rates fluctuate over different months and years.

In [0]:
# Extract Year and Month from DATE OCC
df_day = df_day.withColumn("Year", date_format(col("Date"), "yyyy").cast("int"))
df_day = df_day.withColumn("Month", date_format(col("Date"), "MMMM"))  # Full month name
df_day = df_day.withColumn("MonthNum", date_format(col("Date"), "MM").cast("int"))  # Numeric for sorting

# Count crimes per month and year
crime_trends = df_day.groupBy("Year", "Month", "MonthNum") \
    .agg(count("*").alias("Crime_Count")) \
    .orderBy("Year", "MonthNum")  # Ensure proper chronological order

# Drop MonthNum after sorting (not needed for visualization)
crime_trends = crime_trends.drop("MonthNum")

This cell displays the aggregated crime count per month and year using the Databricks `display()` function.

1. **Displaying the Results**
   - `display(crime_trends)`:  
     - Shows the `crime_trends` DataFrame as an interactive table in Databricks.
     - The table includes:
       - **Year**: The year in which the crime occurred.
       - **Month**: The month name (e.g., "January", "February").
       - **Crime_Count**: The total number of crimes reported in that month.

2. **Insights from the Table**
   - Crime numbers vary month-to-month but generally stay within a certain range.
   - The data is **chronologically sorted** (January → December for each year).
   - This allows for **trend analysis** to identify seasonal variations in crime.

This table serves as the foundation for visualizing crime trends over time.

In [0]:
display(crime_trends)

This cell creates an interactive **line chart** using Plotly to visualize monthly crime trends across different years.

1. **Converting DataFrame to Pandas for Visualization**
   - `crime_trends_pd = crime_trends.toPandas()`:
     - Converts the Spark DataFrame into a Pandas DataFrame.
     - Required because Plotly in Databricks works best with Pandas DataFrames.

2. **Creating a Line Chart to Show Trends**
   - `px.line(...)` generates a line plot with:
     - `x="Month"`: The month names on the x-axis.
     - `y="Crime_Count"`: The total crime frequency for each month.
     - `color="Year"`: Each year's trend is plotted in a separate line.
     - `title="Crime Trends Over Time (Monthly Breakdown)"`: Adds a meaningful title.
     - `labels={"Month": "Month", "Crime_Count": "Crime Frequency", "Year": "Year"}`:
       - Assigns clear labels to axes and legend.
     - `markers=True`: Displays data points on the line for better visibility.

3. **Customizing the Chart Layout**
   - `fig.update_layout(...)` applies additional formatting:
     - `xaxis_tickangle=-45`: Rotates the month labels by **45 degrees** for better readability.
     - `template="plotly_dark"`: Uses a dark theme for Databricks compatibility.

4. **Displaying the Interactive Chart**
   - `fig.show()`: Renders the visualization.

### **Insights from the Output**
- The plot shows **seasonal crime patterns** across multiple years.
- Certain months have noticeable dips or spikes in crime frequency.
- Crime counts appear **relatively stable** across the years, but some years exhibit declining or increasing trends.
- The lowest crime numbers in some years (e.g., 2024) may indicate **external factors** (e.g., policy changes, social conditions, pandemic impact).

This visualization helps in **identifying patterns**, **predicting future trends**, and **assisting law enforcement planning**.

In [0]:
# Convert to Pandas for visualization
crime_trends_pd = crime_trends.toPandas()

# Create a line chart for trends
fig = px.line(
    crime_trends_pd,
    x="Month",
    y="Crime_Count",
    color="Year",  # Different lines for each year
    title="Crime Trends Over Time (Monthly Breakdown)",
    labels={"Month": "Month", "Crime_Count": "Crime Frequency", "Year": "Year"},
    markers=True  # Adds data points on the line
)

# Customize layout
fig.update_layout(
    xaxis_tickangle=-45,  # Rotate labels for readability
    template="plotly_dark",  # Dark theme for Databricks compatibility
)

# Show interactive Plotly graph
fig.show()

This cell aggregates crime data on a **yearly basis** and visualizes the overall trend using a line chart.

1. **Extracting the Year from** `df_day`
   - `df_day.withColumn("Year", date_format(col("Date"), "yyyy").cast("int"))`:
     - Extracts the **year** from the "Date" column.
     - Converts it into an integer for easier filtering and sorting.

2. **Filtering Data for Years 2020 - 2024**
   - `df_filtered = df_trend.filter((col("Year") >= 2020) & (col("Year") <= 2024))`:
     - Filters crime records to only include the years **2020 - 2024**.
     - Ensures we focus on recent crime trends.

3. **Counting Crimes Per Year**
   - `crime_trends_yearly = df_filtered.groupBy("Year").agg(count("*").alias("Crime_Count"))`:
     - Groups the data by **year**.
     - Counts the total number of crimes reported in each year.

   - `.orderBy("Year")`:
     - Ensures that the results are **chronologically ordered**.

4. **Converting DataFrame to Pandas for Visualization**
   - `crime_trends_yearly_pd = crime_trends_yearly.toPandas()`:
     - Converts the Spark DataFrame into a Pandas DataFrame.
     - Required for **Plotly visualizations**.

5. **Creating a Line Chart to Show Yearly Trends**
   - `px.line(...)` generates a **line plot** with:
     - `x="Year"`: The years (2020 - 2024) on the x-axis.
     - `y="Crime_Count"`: The total crime frequency on the y-axis.
     - `title="Crime Trends Over the Years (2020 - 2024)"`: Adds a meaningful title.
     - `labels={"Year": "Year", "Crime_Count": "Crime Frequency"}`:
       - Ensures clear axis labels.
     - `markers=True`: Displays data points on the line.
     - `line_shape="linear"`: Ensures a straight-line connection between data points.

6. **Customizing the Chart Layout**
   - `fig.update_layout(...)` applies additional formatting:
     - `template="plotly_dark"`: Uses a dark theme for Databricks compatibility.
     - `xaxis=dict(tickmode="linear", dtick=1)`: Ensures **each year is labeled** on the x-axis.

7. **Displaying the Interactive Chart**
   - `fig.show()`: Renders the visualization.

### **Insights from the Output**
- **Crime rates increased from 2020 to 2022**, indicating a **rising trend**.
- **2023 saw a slight decline**, suggesting possible external factors (e.g., policy changes, law enforcement measures).
- **2024 shows a sharp drop**, but this might be due to incomplete data for the year or other external influences.
- This visualization helps in **understanding long-term trends** and predicting future crime patterns.

By visualizing yearly crime trends, law enforcement agencies can **analyze patterns**, **allocate resources effectively**, and **identify potential causes** of crime fluctuations.

In [0]:
# Extract Year from DATE OCC
df_trend = df_day.withColumn("Year", date_format(col("Date"), "yyyy").cast("int"))

# Filter data for years 2020-2024
df_filtered = df_trend.filter((col("Year") >= 2020) & (col("Year") <= 2024))

# Count crimes per year
crime_trends_yearly = df_filtered.groupBy("Year") \
    .agg(count("*").alias("Crime_Count")) \
    .orderBy("Year")  # Ensure chronological order

# Convert to Pandas for visualization
crime_trends_yearly_pd = crime_trends_yearly.toPandas()

# Create a line chart for yearly trends
fig = px.line(
    crime_trends_yearly_pd,
    x="Year",
    y="Crime_Count",
    title="Crime Trends Over the Years (2020 - 2024)",
    labels={"Year": "Year", "Crime_Count": "Crime Frequency"},
    markers=True,  # Adds data points on the line
    line_shape="linear"
)

# Customize layout
fig.update_layout(
    template="plotly_dark",  
    xaxis=dict(tickmode="linear", dtick=1)  # Ensure each year is labeled
)

# Show interactive Plotly graph
fig.show()

In [0]:
%pip install folium

# This line uses Databricks' magic command %pip to install the folium Python library within the current notebook's environment. folium is a library used for creating interactive maps.

In [0]:
folder_path = "/FileStore/tables/"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = "," 

df = spark.read.format("csv") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(folder_path)

# Extract the first row as header
new_header = df.first()  # Get first row
df = df.filter(df["_c0"] != new_header[0])  # Remove first row

# Rename columns using extracted header
df = df.toDF(*[str(col) for col in new_header])


display(df)

File Location and Type: It defines the path to the CSV file (file_location) and specifies the file type as "csv" (file_type).
CSV Options: It sets options for reading the CSV: inferSchema is set to "true" to automatically determine the data types of each column, first_row_is_header is set to "true" to treat the first row as column headers, and delimiter is set to "," to specify the comma as the column separator.
Reading the Data: It uses spark.read.format(file_type) to create a DataFrame reader, applies the specified options, and then loads the data from the given file_location into a DataFrame named df.
Displaying the DataFrame: Finally, display(df) is used to display the contents of the DataFrame within the Databricks notebook.

In [0]:
import pandas as pd

# Convert Spark DataFrame to pandas
pandas_df = df.toPandas()

print(pandas_df.shape, "Shape of the dataframe (Number of Observations, Number of Attributes)")
#print(pandas_df.columns)


print(pandas_df.dtypes) # show each attribute's name and data type

# Check for NaN values in each column
nan_counts = pandas_df.isnull().sum()
print('Number of NaN values for each variable:\n',nan_counts)

Spark to pandas Conversion: Converts the Spark DataFrame df into a pandas DataFrame, enabling pandas-specific data manipulation.
DataFrame Shape: Displays the dimensions of the pandas DataFrame (number of rows and columns).
Data Types: Prints the data type of each column in the DataFrame.
NaN Value Check: Calculates the number of missing (NaN) values for each column, and displays these counts.

In [0]:
# Convert "Date Rptd" to datetime format
pandas_df["Date Rptd"] = pd.to_datetime(pandas_df["Date Rptd"], format="%m/%d/%Y %I:%M:%S %p")

# Create a new column with only the date (without time)
pandas_df["Date Rptd2"] = pandas_df["Date Rptd"].dt.date

# Display the first few rows to verify
print(pandas_df[["Date Rptd", "Date Rptd2"]].head())

# Print the range of dates
date_min = pandas_df["Date Rptd2"].min()
date_max = pandas_df["Date Rptd2"].max()

print(f"Date range in 'Date Rptd2': {date_min} to {date_max}")

Datetime Conversion: Converts the "Date Rptd" column, which contains date and time information as strings, into pandas datetime objects. The format argument specifies the expected string format.
Date Extraction: Creates a new column "Date Rptd2" that contains only the date part of the "Date Rptd" column (without the time).
Verification: Displays the first few rows of the "Date Rptd" and "Date Rptd2" columns to ensure the conversion and extraction were successful.
Date Range: Determine the minimum and maximum dates in the "Date Rptd2" column, and prints the range of dates in the dataset.

In [0]:
import folium
from folium.plugins import HeatMap

# Filter data for 2020
pandas_df_2020 = pandas_df[pandas_df["Date Rptd"].dt.year == 2020]

pandas_df_2020["LAT"] = pd.to_numeric(pandas_df_2020["LAT"], errors="coerce")
pandas_df_2020["LON"] = pd.to_numeric(pandas_df_2020["LON"], errors="coerce")

# Ensure the DataFrame contains valid coordinates
pandas_df_2020 = pandas_df_2020.dropna(subset=["LAT", "LON"])

# Create the map using the 2020 crime data
m = folium.Map(location=[pandas_df_2020["LAT"].mean(), pandas_df_2020["LON"].mean()], zoom_start=10)

# Add heatmap for 2020 data
HeatMap(pandas_df_2020[["LAT", "LON"]].values, radius=5, blur=10).add_to(m)

# Display map
m


In [0]:
# Filter data for 2021
pandas_df_2021 = pandas_df[pandas_df["Date Rptd"].dt.year == 2021]

pandas_df_2021["LAT"] = pd.to_numeric(pandas_df_2021["LAT"], errors="coerce")
pandas_df_2021["LON"] = pd.to_numeric(pandas_df_2021["LON"], errors="coerce")
# Ensure the DataFrame contains valid coordinates
pandas_df_2021 = pandas_df_2021.dropna(subset=["LAT", "LON"])

# Create the map using the 2021 crime data
m = folium.Map(location=[pandas_df_2021["LAT"].mean(), pandas_df_2021["LON"].mean()], zoom_start=10)

# Add heatmap for 2021 data
HeatMap(pandas_df_2021[["LAT", "LON"]].values, radius=5, blur=10).add_to(m)

# Display map
m

This code block generates a heatmap visualization of crime data for the year 2021 using folium. It removes any rows that have missing (NaN) values in the "LAT" (latitude) or "LON" (longitude) columns, ensuring that only valid coordinates are used for the map. It creates a folium.Map object where the center location is set to the mean latitude and longitude of the filtered 2021 data, and sets an initial zoom level. Within the heatmap, we can set the radius of each heatmap point and the blur level.

In [0]:
# Count the occurrences of each crime type
crime_counts = pandas_df["Crm Cd Desc"].value_counts()

# Get the top 10 most frequent crime types
top_10_crime_types = crime_counts.head(10)

# Display the frequency table
print(top_10_crime_types)

This code calculates and displays the top 10 most frequent crime types from the "Crm Cd Desc" column of the pandas DataFrame.

In [0]:
# Calculate the percentage of total crimes for each category
total_crimes = len(pandas_df)
top_10_crime_percentages = (top_10_crime_types / total_crimes) * 100

# Combine the counts and percentages into a DataFrame for better display
result_df = pd.DataFrame({
    'Count': top_10_crime_types,
    'Percentage': top_10_crime_percentages
})

# Display the frequency table with percentages
print(result_df)

In [0]:
# Filter data for vehicle stolen crimes
pandas_df_vehicle = pandas_df[pandas_df["Crm Cd Desc"] == 'VEHICLE - STOLEN']

pandas_df_vehicle["LAT"] = pd.to_numeric(pandas_df_vehicle["LAT"], errors="coerce")
pandas_df_vehicle["LON"] = pd.to_numeric(pandas_df_vehicle["LON"], errors="coerce")
# Ensure the DataFrame contains valid coordinates
pandas_df_vehicle = pandas_df_vehicle.dropna(subset=["LAT", "LON"])

# Create the map based on vehicle stolen crimes
m = folium.Map(location=[pandas_df_vehicle["LAT"].mean(), pandas_df_vehicle["LON"].mean()], zoom_start=10)

# Add heatmap
HeatMap(pandas_df_vehicle[["LAT", "LON"]].values, radius=5, blur=10).add_to(m)

# Display map
m

Repeat the previous folium map code, but this time we filter on observations in the dataset for stolen vehicle crimes.

In [0]:
# Filter data for battery crimes
pandas_df_battery = pandas_df[pandas_df["Crm Cd Desc"] == 'BATTERY - SIMPLE ASSAULT']

pandas_df_battery["LAT"] = pd.to_numeric(pandas_df_battery["LAT"], errors="coerce")
pandas_df_battery["LON"] = pd.to_numeric(pandas_df_battery["LON"], errors="coerce")
# Ensure the DataFrame contains valid coordinates
pandas_df_battery = pandas_df_battery.dropna(subset=["LAT", "LON"])

# Create the map based on battery crimes
m = folium.Map(location=[pandas_df_battery["LAT"].mean(), pandas_df_battery["LON"].mean()], zoom_start=10)

# Add heatmap
HeatMap(pandas_df_battery[["LAT", "LON"]].values, radius=5, blur=10).add_to(m)

# Display map
m

Repeat the previous folium map code, but this time we filter on observations in the dataset for battery crimes.

In [0]:
# Filter data for intimate partner assult
pandas_df_partner = pandas_df[pandas_df["Crm Cd Desc"] == 'INTIMATE PARTNER - SIMPLE ASSAULT']

pandas_df_partner["LAT"] = pd.to_numeric(pandas_df_partner["LAT"], errors="coerce")
pandas_df_partner["LON"] = pd.to_numeric(pandas_df_partner["LON"], errors="coerce")
# Ensure the DataFrame contains valid coordinates
pandas_df_partner = pandas_df_partner.dropna(subset=["LAT", "LON"])

# Create the map based on intimate partner assault
m = folium.Map(location=[pandas_df_partner["LAT"].mean(), pandas_df_partner["LON"].mean()], zoom_start=10)

# Add heatmap
HeatMap(pandas_df_partner[["LAT", "LON"]].values, radius=5, blur=10).add_to(m)

# Display map
m

Repeat the previous folium code, but this time we filter on observations in the dataset for intimate partner assault crimes.